Web Scraping by using BeautifulSoup

In [ ]:
import pandas as pd
import bs4
import requests
import openpyxl

In [ ]:
data = requests.get('https://southernjewelrynews.com/?s=sustainable+material')

In [ ]:
soup = bs4.BeautifulSoup(data.text)
soup

In [ ]:
title = soup.find('article',{'class':'jeg_post jeg_pl_md_2 format-standard'})

In [ ]:
title.find('h3',{'class':'jeg_post_title'}).text.replace('\n','')

In [ ]:
desc = title.find('div',{'class':'jeg_post_excerpt'}).text.replace('\n','')

In [ ]:
len(soup.find_all('article',{'class':'jeg_post jeg_pl_md_2 format-standard'}))

In [ ]:
i_page = 1
title_list = []
cats_list = []
desc_list = []
while i_page <= 3:
  data = requests.get(f'https://southernjewelrynews.com/page/{i_page}/?s=sustainable+material')
  soup = bs4.BeautifulSoup(data.text)
  for c in soup.find_all('article', class_='jeg_post jeg_pl_md_2 format-standard'):
    title_list.append(c.find('h3',{'class':'jeg_post_title'}).text.replace('\n',''))
    desc_list.append(c.find('div', class_='jeg_post_excerpt').text.strip())
  print('Complete page number: ',i_page)
  i_page += 1

In [ ]:
table = pd.DataFrame([title_list,desc_list]).transpose()
table.columns = ['title','description']
table.set_index('title')

In [ ]:
soup.find('div', class_= 'jeg_postblock_content').text

In [ ]:
df = pd.DataFrame(table)

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df.info()

In [ ]:
df.drop_duplicates(inplace=True)
df

In [ ]:
df2 = df.drop([19])
df2

In [ ]:
df3 =  df2.replace({'\®':'','\’':'\'','\“':'\"','\”':'\"','–':'-','\s{2,}':' '}, regex = True)
df3

In [ ]:
df3.to_csv(r'C:\Users\acer\Desktop\newsinsouthernjewelrynews.csv',index=False)

Uploaded csv file to DataPlatform and Covert file to parquet

In [ ]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
dir_raw_id = 45
dir_process_id = 123

In [ ]:
datanode = DataNode()

In [ ]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='newsinsouthernjewelrynews.csv'))
meta

In [ ]:
df = pd.read_csv(fp)
df

In [ ]:
datanode.write(df=df, directory=dir_process_id, name="sustainable_material_in_southernjewelrynews", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='newsinsouthernjewelrynews.csv')])